# ENGR 240: Stiff ODEs and Numerical Instability

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%209/Worksheet%209-2%20Stiff%20ODEs%20Chemical%20Reactor.ipynb)

## Introduction

**Stiff ODEs** are a special class of differential equations where the solution contains both fast-decaying and slow-varying components. These systems pose significant challenges for numerical methods because explicit methods (like Euler or RK45) require extremely small step sizes to maintain stability, making them computationally expensive or even impossible to solve.

### Chemical Reactor Example: Fast Reactions with Different Timescales

Consider a chemical reactor with two competing reactions:
- **Fast reaction**: $A \rightarrow B$ (very rapid equilibrium)
- **Slow reaction**: $B \rightarrow C$ (slow conversion to final product)

The system of ODEs describing the concentrations is:

$$\frac{d[A]}{dt} = -k_1[A]$$
$$\frac{d[B]}{dt} = k_1[A] - k_2[B]$$
$$\frac{d[C]}{dt} = k_2[B]$$

**Stiffness arises when**: $k_1 \gg k_2$ (fast reaction much faster than slow reaction)

**Engineering Relevance:**
- **Chemical reactors**: Fast pre-equilibrium followed by slow rate-determining step
- **Combustion modeling**: Fast fuel oxidation with slow heat transfer
- **Circuit analysis**: Fast capacitor charging with slow RC time constants
- **Population dynamics**: Fast birth/death with slow environmental changes

**Learning Objectives:**
- Understand what makes an ODE system "stiff"
- Compare explicit vs implicit numerical methods on stiff systems
- Demonstrate numerical instability with inappropriate methods
- Learn when to use different scipy.integrate.solve_ivp methods
- Visualize the timescale separation that causes stiffness

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import time
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
print("Libraries imported successfully!")

## Task 1: Implement the Chemical Reactor System

In [ ]:
def chemical_reactor(t, concentrations, k1, k2):
    """
    Chemical reactor with two sequential reactions: A -> B -> C
    
    Parameters:
    t: time
    concentrations: [A, B, C] concentration vector
    k1: rate constant for A -> B (fast reaction)
    k2: rate constant for B -> C (slow reaction)
    
    Returns:
    derivatives: [dA/dt, dB/dt, dC/dt]
    """
    A, B, C = concentrations
    
    dA_dt = -k1 * A
    dB_dt = k1 * A - k2 * B
    dC_dt = k2 * B
    
    return [dA_dt, dB_dt, dC_dt]

# Test the function
test_derivs = chemical_reactor(0, [1.0, 0.0, 0.0], 100, 1)
print(f"Test derivatives at t=0: {test_derivs}")
print(f"Mass balance check (should be ~0): {sum(test_derivs)}")

## Task 2: Compare Non-Stiff vs Stiff Systems

Let's first solve a **non-stiff** system where reaction rates are similar, then a **stiff** system where they differ by orders of magnitude.

In [ ]:
# Initial conditions: Start with pure A
y0 = [1.0, 0.0, 0.0]  # [A, B, C]
t_span = (0, 5)  # 5 seconds
t_eval = np.linspace(0, 5, 1000)

# Case 1: Non-stiff system (similar reaction rates)
k1_normal = 2.0   # Fast reaction rate
k2_normal = 1.0   # Slow reaction rate
print(f"Non-stiff case: k1/k2 ratio = {k1_normal/k2_normal}")

# Case 2: Stiff system (very different reaction rates) - EXTREMELY STIFF
k1_stiff = 500000.0  # Half million reaction rate - EXTREME
k2_stiff = 1.0       # Slow reaction rate
print(f"Stiff case: k1/k2 ratio = {k1_stiff/k2_stiff}")

# Solve both systems using RK45 (explicit method)
print("\nSolving non-stiff system with RK45...")
sol_normal = solve_ivp(chemical_reactor, t_span, y0, 
                      args=(k1_normal, k2_normal), 
                      t_eval=t_eval, method='RK45', rtol=1e-6)

print("Solving stiff system with RK45...")
sol_stiff_rk45 = solve_ivp(chemical_reactor, t_span, y0, 
                          args=(k1_stiff, k2_stiff), 
                          t_eval=t_eval, method='RK45', rtol=1e-6)

print(f"Non-stiff solution: {sol_normal.nfev} function evaluations")
print(f"Stiff solution (RK45): {sol_stiff_rk45.nfev} function evaluations")
print(f"Success: Normal={sol_normal.success}, Stiff={sol_stiff_rk45.success}")

In [ ]:
# Plot the comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Non-stiff system
ax1.plot(sol_normal.t, sol_normal.y[0], 'b-', label='[A]', linewidth=2)
ax1.plot(sol_normal.t, sol_normal.y[1], 'r-', label='[B]', linewidth=2)
ax1.plot(sol_normal.t, sol_normal.y[2], 'g-', label='[C]', linewidth=2)
ax1.set_title(f'Non-Stiff System (k₁={k1_normal}, k₂={k2_normal})')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Concentration (mol/L)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Non-stiff system - zoomed view of early times
mask_early = sol_normal.t <= 2.0
ax2.plot(sol_normal.t[mask_early], sol_normal.y[0][mask_early], 'b-', label='[A]', linewidth=2)
ax2.plot(sol_normal.t[mask_early], sol_normal.y[1][mask_early], 'r-', label='[B]', linewidth=2)
ax2.plot(sol_normal.t[mask_early], sol_normal.y[2][mask_early], 'g-', label='[C]', linewidth=2)
ax2.set_title('Non-Stiff: Early Time Detail')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Concentration (mol/L)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Stiff system
if sol_stiff_rk45.success:
    ax3.plot(sol_stiff_rk45.t, sol_stiff_rk45.y[0], 'b-', label='[A]', linewidth=2)
    ax3.plot(sol_stiff_rk45.t, sol_stiff_rk45.y[1], 'r-', label='[B]', linewidth=2) 
    ax3.plot(sol_stiff_rk45.t, sol_stiff_rk45.y[2], 'g-', label='[C]', linewidth=2)
    ax3.set_title(f'Stiff System - RK45 (k₁={k1_stiff}, k₂={k2_stiff})')
else:
    ax3.text(0.5, 0.5, 'RK45 FAILED\nfor Stiff System', 
             ha='center', va='center', transform=ax3.transAxes, 
             fontsize=16, color='red', weight='bold')
    ax3.set_title('Stiff System - RK45 FAILURE')

ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Concentration (mol/L)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Stiff system - early time detail (if solution exists)
if sol_stiff_rk45.success and len(sol_stiff_rk45.t) > 10:
    # Show first 0.01 seconds to see the fast transient
    mask_very_early = sol_stiff_rk45.t <= 0.01
    if np.any(mask_very_early):
        ax4.plot(sol_stiff_rk45.t[mask_very_early], sol_stiff_rk45.y[0][mask_very_early], 'b-', label='[A]', linewidth=2)
        ax4.plot(sol_stiff_rk45.t[mask_very_early], sol_stiff_rk45.y[1][mask_very_early], 'r-', label='[B]', linewidth=2)
        ax4.plot(sol_stiff_rk45.t[mask_very_early], sol_stiff_rk45.y[2][mask_very_early], 'g-', label='[C]', linewidth=2)
        ax4.set_title('Stiff: Fast Transient (first 0.01s)')
    else:
        ax4.text(0.5, 0.5, 'No early time\ndata available', 
                 ha='center', va='center', transform=ax4.transAxes)
        ax4.set_title('Stiff: Early Time (No Data)')
else:
    ax4.text(0.5, 0.5, 'Solution Failed\nor Insufficient Data', 
             ha='center', va='center', transform=ax4.transAxes, 
             fontsize=14, color='red')
    ax4.set_title('Stiff: Early Time Detail - Failed')

ax4.set_xlabel('Time (s)')
ax4.set_ylabel('Concentration (mol/L)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print(f"1. Non-stiff system: Smooth, gradual transitions")
print(f"2. Stiff system: Very rapid initial change in [A] and [B], then slow evolution")
if not sol_stiff_rk45.success:
    print(f"3. RK45 struggles with stiff systems - may fail or require tiny steps!")

## Task 3: Demonstrate Solver Failure and Success

Now let's compare different numerical methods on the stiff system to see which ones succeed and which fail.

In [ ]:
# Test different numerical methods on the stiff system
methods = {
    'RK45': 'Explicit Runge-Kutta (4th/5th order)',
    'RK23': 'Explicit Runge-Kutta (2nd/3rd order)', 
    'DOP853': 'Explicit Runge-Kutta (8th order)',
    'Radau': 'Implicit Runge-Kutta (5th order) - STIFF SOLVER',
    'BDF': 'Backward Differentiation Formula - STIFF SOLVER',
    'LSODA': 'Adams-Bashforth-Moulton/BDF with stiffness detection'
}

solutions = {}
performance = {}

print("Testing different numerical methods on stiff system...\n")

for method_name, description in methods.items():
    print(f"Trying {method_name}: {description}")
    
    try:
        # Time the solution
        start_time = time.time()
        sol = solve_ivp(chemical_reactor, t_span, y0,
                       args=(k1_stiff, k2_stiff),
                       t_eval=t_eval, method=method_name, 
                       rtol=1e-6, atol=1e-9)
        end_time = time.time()
        solve_time = end_time - start_time
        
        solutions[method_name] = sol
        performance[method_name] = {
            'success': sol.success,
            'nfev': sol.nfev,
            'njev': getattr(sol, 'njev', 0),  # Jacobian evaluations
            'nlu': getattr(sol, 'nlu', 0),    # LU decompositions
            'time': solve_time,
            'message': sol.message
        }
        
        if sol.success:
            print(f"  ✅ SUCCESS: {sol.nfev} function evaluations in {solve_time:.3f}s")
        else:
            print(f"  ❌ FAILED: {sol.message}")
            
    except Exception as e:
        print(f"  ❌ ERROR: {str(e)}")
        performance[method_name] = {'success': False, 'error': str(e), 'time': 0}
    
    print()

# Create performance summary table with timing
print("\n" + "="*90)
print("PERFORMANCE SUMMARY WITH TIMING")
print("="*90)
print(f"{'Method':<10} {'Success':<8} {'Time(s)':<8} {'Func Evals':<12} {'Jacobian':<10} {'LU Decomp':<10} {'Status'}")
print("-"*90)

for method_name in methods.keys():
    perf = performance[method_name]
    if 'error' in perf:
        print(f"{method_name:<10} {'ERROR':<8} {'-':<8} {'-':<12} {'-':<10} {'-':<10} {perf['error'][:25]}")
    else:
        status = "✅" if perf['success'] else "❌"
        time_str = f"{perf['time']:.3f}" if perf['success'] else "-"
        print(f"{method_name:<10} {status:<8} {time_str:<8} {perf['nfev']:<12} {perf['njev']:<10} {perf['nlu']:<10} {perf['message'][:25]}")

In [ ]:
# Plot results from successful methods
successful_methods = [name for name, perf in performance.items() 
                     if perf.get('success', False)]

if successful_methods:
    n_success = len(successful_methods)
    fig, axes = plt.subplots(2, (n_success + 1) // 2, figsize=(15, 10))
    if n_success == 1:
        axes = [axes]
    elif axes.ndim == 1:
        axes = axes.reshape(1, -1)
    
    axes_flat = axes.flatten()
    
    for i, method in enumerate(successful_methods):
        sol = solutions[method]
        ax = axes_flat[i]
        
        ax.plot(sol.t, sol.y[0], 'b-', label='[A]', linewidth=2)
        ax.plot(sol.t, sol.y[1], 'r-', label='[B]', linewidth=2)
        ax.plot(sol.t, sol.y[2], 'g-', label='[C]', linewidth=2)
        
        time_str = f"{performance[method]['time']:.3f}s"
        ax.set_title(f'{method}\n{performance[method]["nfev"]} evals, {time_str}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Concentration (mol/L)')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    # Hide unused subplots
    for j in range(len(successful_methods), len(axes_flat)):
        axes_flat[j].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey Insights:")
    print("• Explicit methods may require massive computational resources on stiff systems")
    print("• Implicit methods (Radau, BDF) are designed for stiff systems")
    print("• LSODA automatically detects stiffness and switches methods")
    print("• Notice the dramatic difference in function evaluations and timing!")
else:
    print("No methods succeeded! This demonstrates the extreme challenge of stiff systems.")

## Task 4: TODO - Explore Step Size Effects

**Your turn!** Test how different maximum step sizes affect RK45 stability on stiff systems.

In [ ]:
# TODO: Test RK45 with different maximum step sizes
step_sizes = [0.1, 0.01, 0.001, 0.0001]

print("Testing RK45 with different maximum step sizes on stiff system...\n")

plt.figure(figsize=(16, 8))

for i, max_step in enumerate(step_sizes):
    print(f"Trying max_step = {max_step}...")
    
    # TODO: Complete this - use solve_ivp with max_step parameter
    # sol = solve_ivp(chemical_reactor, t_span, y0,
    #                args=(k1_stiff, k2_stiff),
    #                method='RK45', max_step=???, rtol=1e-6)
    
    # TODO: Plot results in subplot (2,2,i+1)
    # TODO: Add success/failure indicators
    
    pass  # Remove this when you add your code

plt.tight_layout()
plt.show()

print("\nQuestion: What happens to computational cost as step size decreases?")

## Task 5: TODO - Create Your Own Stiff System

**Challenge:** Modify the reaction rates to create different levels of stiffness.

In [ ]:
# TODO: Experiment with different stiffness ratios
ratios_to_test = [10, 100, 1000, 10000, 100000]

# TODO: For each ratio, test both RK45 and BDF
# TODO: Record which ratios cause RK45 to fail or become extremely slow
# TODO: Plot the computational cost vs stiffness ratio

print("TODO: Find the stiffness ratio where RK45 becomes impractical!")

# Hint: Use a loop to test each ratio
# Hint: Store success/failure results and timing in a list or dictionary
# Hint: Create a log plot showing computational cost vs stiffness ratio

## Discussion Questions

1. **What makes an ODE system "stiff"?**

2. **Why do explicit methods fail on stiff systems?**

3. **When should you use implicit methods in engineering applications?**

4. **How can you detect stiffness in practice?**

## Key Takeaways

### Solver Selection Guidelines:
- **Non-stiff systems**: Use RK45 or RK23 (explicit methods)
- **Stiff systems**: Use BDF or Radau (implicit methods) 
- **Unknown stiffness**: Use LSODA (adaptive switching)
- **Rule of thumb**: If explicit methods are very slow, try implicit methods

### Engineering Applications:
- **Chemical reactors**: Fast pre-equilibrium + slow main reaction
- **Combustion**: Fast ignition + slow heat transfer
- **Electronics**: Fast switching + slow RC charging
- **Biology**: Fast enzyme kinetics + slow population dynamics

Understanding stiffness is crucial for efficient simulation of real engineering systems!